# Lecture 7 Advanced Python Topics: Decorators

- [7.1 Introduction](#section1)
- [7.2 Why Use Decorators](#section2)
- [7.3 Other Properties of Decorators](#section3)
- [7.4 Decorator Examples](#section4)
- [7.5 Class Decorators](#section5)
- [References](#section6)

# 7.1 Introduction <a id="section1"/>

> A ***decorator*** is a design pattern in Python that allows to add new functionality to an existing object without modifying its structure. Decorators are typically called before the definition of a function that is to be decorated by using the syntax @decorator.

Using decorators is also called *metaprogramming* because a part of the program tries to modify another part of the program at execution time.

Let's consider the following simple function `divide`, which accepts two arguments `a` and `b`. We know it will give an error if we pass in `b` as 0.

In [1]:
def divide(a, b):
    return a/b

In [2]:
divide(12,3)

4.0

In [3]:
divide(2,0)

ZeroDivisionError: division by zero

Let's assume that we would like to modify the function `divide` so that it checks for the case of division by 0 that will cause the error.

One way to achieve this is to create a new function that will take as an argument the function `divide` and modify it. This way, we can apply the same functionality to other similar functions as well, if we needed to. 

The following function named `smart_divide` does exactly that.

In [4]:
def smart_divide(func):
    def inner_function(a, b):
        print("Divide", a, "and", b)
        if b == 0:
            print("Cannot divide by 0")
            return
        return func(a, b)
    return inner_function

Now, we can use the function `divide` as an argument in the new function `smart_divide`. We can name the new function `my_divide`.

In [5]:
my_divide = smart_divide(divide)

In [6]:
my_divide(12,3)

Divide 12 and 3


4.0

In [7]:
my_divide(12,0)

Divide 12 and 0
Cannot divide by 0


Modifying existing functions using a decorator is a common construct in Python, and the syntax uses the `@` symbol along with the name of the decorator function placed above the definition of the function to be decorated. 

In [8]:
@smart_divide
def divide(a, b):
    print(a/b)

In [9]:
divide(12,3)

Divide 12 and 3
4.0


In [10]:
divide(12,0)

Divide 12 and 0
Cannot divide by 0


Using `@smartdivide` in the above code is equivalent to writing `divide = smart_divide(divide)`.

The new functionality added by the decorator function `smart_divide` to the original function `divide` can be seen as similar to packing a gift, where the decorator acts as a wrapper. The actual gift inside the wrapper does not alter, but now it looks pretty since it got decorated.

Or, in other words, the decorator is a function that modifies another function. When the decorated function is invoked through its original name `divide`, the decorator is applied to augment the original function in some way. 

#### Example 2

One more simple example follows, where the function `ordinary()` does not take any input arguments. 

In [11]:
def ordinary():
    print("Ordinary function")
    
ordinary() 

Ordinary function


A decorator function called `make_pretty()` wraps around the code of the argument function `func_1` and inserts additional print statements before and after the function `func_1`. 

In [12]:
def make_pretty(func_1):    # the input argument to the decorator is another function, here called 'func_1'
    def inner():            # this is the wrapper for the function 
        print("The function will be decorated")
        func_1()            # this executes the initial function
        print("The function was decorated")
    return inner          

Let's apply the decorator, and call the function `ordinary()` after it has been decorated.

In [13]:
@make_pretty
def ordinary():
    print("Ordinary function")
    
ordinary()

The function will be decorated
Ordinary function
The function was decorated


In practice, we don't even need to ever use decorators, and we can achieve the same results by just using `make_pretty(ordinary)` as shown below. However, decorators have advantages over such approach, and are commonly used. 

In [14]:
def ordinary():
    print("Ordinary function")

ordinary = make_pretty(ordinary)

ordinary()

The function will be decorated
Ordinary function
The function was decorated


### Functions as Arguments, Inner Functions, and Returns in Other Functions

In order to better understand decorators, we will now take a step back and explain several related concepts in Python.

As we mentioned before, everything in Python is an ***object***. Names that we assign to objects are simply identifiers bound to these objects. Functions are no exceptions, and they are objects too.

Functions can be passed as arguments to another function. Here is an example, where the functions `increase` and `decrease` are passed as arguments to the function `operate`. Functions like `operate` that take other functions as arguments are also called ***higher order functions***. 

In [15]:
def increase(x):
    # Increse x by 1
    return x + 1


def decrease(x):
    # Decrease x by 1
    return x - 1


def operate(func, x):   # Operate takes 2 arguments: a function `func` and a number `x`
    result = func(x)
    return result

In [16]:
# call 'operate' with 'increase' as argument
operate(increase,3)

4

In [17]:
# call 'operate' with 'decrease' as argument
operate(decrease,3)

2

Furthermore, other functions can be nested under another function, and are referred to as ***inner functions***. 

Here’s an example of a function with two inner functions.

In [18]:
def parent():
    print("Printing from the parent() function")

    def first_child():
        print("Printing from the first_child() function")

    def second_child():
        print("Printing from the second_child() function")

    second_child()
    first_child()

In [19]:
parent()

Printing from the parent() function
Printing from the second_child() function
Printing from the first_child() function


Note that the order in which the inner functions are defined does not matter, and what matter is the order in which the inner functions are called within the `parent()` function. That is, `second_child` is called first, and afterward `first_child` is called.

As we know, the inner functions are in the  local scope to the outer (enclosing) function `parent()`, and they only exist inside the `parent()` function as local variables. If we try calling `first_child()` we will get an error.

In [20]:
first_child()

NameError: name 'first_child' is not defined

Python also allows to use other functions as ***returns*** of a function. The following example returns one of the inner functions from the outer `parent()` function, based on an `if` test.

In [21]:
def parent(num):
    def first_child():
        return "Hi, I am Emma"

    def second_child():
        return "Call me Liam"

    if num == 1:
        return first_child
    else:
        return second_child

In [22]:
my_func1 = parent(1)
my_func1()

'Hi, I am Emma'

In [23]:
my_func2 = parent(5)
my_func2()

'Call me Liam'

In [24]:
type(my_func2)

function

In [25]:
# my_func_2 is a function object
my_func2

<function __main__.parent.<locals>.second_child()>

Also note that in the previous example we executed the inner functions within the parent function, because we used `first_child()` and `second_child()`. However, in this last example, we did not add parentheses to the inner functions `first_child` and `second_child` in the return lines. That way, we obtained a reference to each function that we can later call, that is `my_func1` and `my_func2` above are function objects, and we need to call them with parentheses (e.g., `my_func1()`) in order to obtain the print statement. If we don't use parentheses (as in `my_func2`), Python will just display that this is a function object. 

Compare to the code below, where we called the inner functions in the return lines. 

In [26]:
def parent(num):
    def first_child():
        return "Hi, I am Emma"

    def second_child():
        return "Call me Liam"

    if num == 1:
        return first_child()
    else:
        return second_child()

In [27]:
my_func2 = parent(5)
my_func2

'Call me Liam'

In [28]:
type(my_func2)

str

In [29]:
# my_func2 here is a string and not a function, we cannnot call it
my_func2()

TypeError: 'str' object is not callable

One more similar simple example is shown below where a nested function is returned in an outer function. Each time we call `first_function()`, `nested_function()` is returned.

In [30]:
def first_function():  # created 1st function
    
    print('First line in the called function')
    
    def nested_function():  # Created 2nd function (nested)
        print('Hello, this is inside the nested function')
    return nested_function

In [31]:
new_function = first_function()

new_function()

First line in the called function
Hello, this is inside the nested function


In [32]:
# Compare to the following
first_function()

First line in the called function


<function __main__.first_function.<locals>.nested_function()>

### General Syntax of a Decorator

The following code provides a general syntax of a decorator. The `decorator_function` takes a function called `func` as its argument, and returns a modified version of it called `new_function`. 

    def decorator_function(func):                    # the 'decorator_function' will be later invoked using the @ syntax
        def new_function(*args, **kwargs):           # this function is often named 'wrapper' or 'inner' function
            # Perform actions with `func`, `args` and `kwargs`
            ...

        return new_function

    @decorator_function
    def my_function(arg1, arg2, arg3, ....):         # my_function is passed as an argument to 'decorator_function'
        # Peform some actions, equivalent to 'my_function = decorator_function(my_function)'
   
   
    my_function(arg1=v1, arg2=v2, arg3=v3,....)     # call 'my_function', with the values v1, v2, v3, ... passed to 'new_function'

# 7.2 Why Use Decorators <a id="section2"/>

Like other advanced Python tools, decorators are never strictly required from a purely technical perspective: we can implement the same functionality using simple helper functions or other techniques. Or, we can always manually
modify the code in a function instead of using a decorator.

However, imagine the scenario where instead of adding some functionality to one function, we have a large package with hundreds of functions to which we would like to add the same functionality. To do that, we would need to copy the same thing over and over again, which is error-prone, as we could miss one place where it is required, or paste it in the wrong place. Using decorators allows to modify all functions in a manner that is less error-prone.

There are also other reasons for using decorators that go beyond just avoiding repetitive typing. When we need to modify the logic in our programs, we can modify the logic in just one place, instead of trying to find related code everywhere, and perhaps make mistakes along the way.

Furthermore, decorators have a very explicit syntax with the @ symbol, which makes them easier to spot than helper
function calls that may be arbitrarily far-removed from the functions upon which they act.

In summary, decorators offer advantages in terms of both code maintenance and consistency. Although the choice to use decorators is still subjective, their advantages are compelling enough so that they are adopted by many Python users.

# 7.3 Other Properties of Decorators <a id="section3"/>

### Chaining Decorators

Multiple decorators can be chained in Python, where a function can be decorated multiple times with different (or same) decorators. This is achieved by listing the decorators above the desired function.

In [33]:
def star(func):
    def inner():
        print('Star decorator wrapper')
        print('*' * 30)
        func()
        print('Star decorator exit')
    return inner


def percent(func):
    def inner():
        print('Percent decorator wrapper')
        print("%" * 30)
        func()
        print('Percent decorator exit')
    return inner


@star
@percent
def say_hello():
    print('Hello')

In [34]:
# this is equivalent to 'say_hello =  star(percent(say_hello))'
say_hello()

Star decorator wrapper
******************************
Percent decorator wrapper
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Hello
Percent decorator exit
Star decorator exit


The order in which the decorators are chained matters. If we had reversed the order, the output is different.

In [35]:
@percent
@star
def say_hello():
    print('Hello')

In [36]:
say_hello()

Percent decorator wrapper
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Star decorator wrapper
******************************
Hello
Star decorator exit
Percent decorator exit


### Reusing Decorators

Since a decorator is just a regular Python function, all the usual tools for easy reusability are available. 

Let’s move the decorators `star` and `percent` to their own module called `my_decorators.py` that can be used in  many other functions.

We can now use these decorators in other files by doing a regular import.

In [37]:
from my_decorators import star, percent

@star
@percent
def say_hello():
    print('Hello')
    
say_hello()

Star decorator wrapper
******************************
Percent decorator wrapper
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Hello
Percent decorator exit
Star decorator exit


### Decorating Functions With Arguments

If the decorated function needed to accept some arguments, the above decorators will not work. The problem is that the inner wrapper function `inner()` does not take any arguments. 

In the next example, passing the string "Hi there" raised an error stating that `inner()` takes 0 arguments. In other words, `inner()`needs to accept the same arguments as the decorated function `say_hello()`. 

In [38]:
def percent(func):
    def inner():
        print('Percent decorator wrapper')
        print("%" * 30)
        func()
        print('Percent decorator exit')
    return inner

@percent
def say_hello(msg):
    print(msg)

In [39]:
say_hello('Hi there')

TypeError: inner() takes 0 positional arguments but 1 was given

We can fix that by rewriting `inner()` to accept an argument, which will correspond to the `msg` argument in the `say_helo()` function. However, there is a more general solution, which uses the syntax `*args` and `**kwargs` in the inner wrapper function. In that case, `inner()` will accept an arbitrary number of positional and keyword arguments. 

Therefore, we can rewrite `star` and `percent` as follows. Now, the inner wrapper functions can accept arbitrary number of arguments. 

In [40]:
def star(func):
    def inner(*args, **kwargs):
        print('Star decorator wrapper')
        print('*' * 30)
        func(*args, **kwargs)
        print('Star decorator exit')
    return inner


def percent(func):
    def inner(*args, **kwargs):
        print('Percent decorator wrapper')
        print("%" * 30)
        func(*args, **kwargs)
        print('Percent decorator exit')
    return inner


@star
@percent
def say_hello(msg1, msg2, msg3='Nice to see you', msg4='Good bye'):
    print(msg1, ':', msg2, ':', msg3, ':', msg4)

In [41]:
say_hello('Hello', "How are you?", msg4='Hi')

Star decorator wrapper
******************************
Percent decorator wrapper
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Hello : How are you? : Nice to see you : Hi
Percent decorator exit
Star decorator exit


### functools Library with Decorators

A great convenience when working with Python is its powerful introspection ability. For instance, for every object, we can find out information about its name, or we can use `help` to read a description of the object. 

Examples are shown below for the built-in `print` function in Python.

In [42]:
print

<function print>

In [43]:
# name of the function
print.__name__

'print'

In [44]:
help(print)

Help on built-in function print in module builtins:

print(...)
    print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
    
    Prints the values to a stream, or to sys.stdout by default.
    Optional keyword arguments:
    file:  a file-like object (stream); defaults to the current sys.stdout.
    sep:   string inserted between values, default a space.
    end:   string appended after the last value, default a newline.
    flush: whether to forcibly flush the stream.



However, these introspection tools don't work as expected for decorated functions. For instance, if we check the information about the decorated function `say_hello()`, we can see that Python reports that the name of the function is `inner()`, which is the wrapper function in the decorator. Although this is technically true, this information is not very helpful.

In [45]:
say_hello

<function __main__.star.<locals>.inner(*args, **kwargs)>

In [46]:
say_hello.__name__

'inner'

In [47]:
help(say_hello)

Help on function inner in module __main__:

inner(*args, **kwargs)



The `functools` library in Python offers a tool to fix that. Using the decorator @functools.wraps from the `functools` library preserves the information about the original function. The updated decorator `star()` is shown below. 

In [48]:
import functools

def star(func):
    @functools.wraps(func)       # only insert this line before the inner function
    def inner(*args, **kwargs):
        print('Star decorator wrapper')
        print('*' * 30)
        func(*args, **kwargs)
        print('Star decorator exit')
    return inner


@star
def say_hello(msg1, msg2):
    'Function that prints a collection of strings'
    print(msg1, ':', msg2)
    
say_hello('Hello', 'How are you?')

Star decorator wrapper
******************************
Hello : How are you?
Star decorator exit


This results in correct name and introspection information about the original function `say_hello()` after the decoration.

In [49]:
say_hello

<function __main__.say_hello(msg1, msg2)>

In [50]:
say_hello.__name__

'say_hello'

In [51]:
help(say_hello)

Help on function say_hello in module __main__:

say_hello(msg1, msg2)
    Function that prints a collection of strings



# 7.4 Decorator Examples <a id="section4"/>

#### Timing Functions

An example of a decorator is `@timer` that measures the time a function takes to execute and print the time. 

In the example below, the decorator measures the time for creating a tuple multiple times.

In [52]:
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.8f} secs")
        return value
    return wrapper_timer

@timer
def create_tuple(num_times):
    for n in range(num_times):
        tuple1 = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)        

In [53]:
# measures the execution time for creating the tuple 1 time
create_tuple(1)

Finished 'create_tuple' in 0.00000220 secs


In [54]:
# measures the execution time for creating the tuple 1000 times
create_tuple(1000)

Finished 'create_tuple' in 0.00007820 secs


#### Authorization

Decorators can help to check whether someone is authorized to use an endpoint in a web application. They are extensively used in web frameworks like Flask and Django (will be studied later in the course). Here is an example to employ decorator-based authentication.

In [55]:
def requires_auth(f):
    @functools.wraps(func)
    def decorated(*args, **kwargs):
        auth = request.authorization
        if not auth or not check_auth(auth.username, auth.password):
            authenticate()
        return f(*args, **kwargs)
    return decorated

#### Logging

Logging is another use case where decorators are often used.

This example just prints the name of the function that 'was called'.

In [57]:
def logit(func):
    @functools.wraps(func)
    def with_logging(*args, **kwargs):
        print(func.__name__ + " was called")
        return func(*args, **kwargs)
    return with_logging

@logit
def addition_func(x):
    # apply some actions on x
    x = x*6
    return x

In [58]:
addition_func(4)

addition_func was called


24

One more example follows, which allows to specify a logfile to output to the name of the called function. Every  function call will be recorded in the logfile.

In [59]:
def logit(logfile='out.log'):
    def logging_decorator(func):
        @functools.wraps(func)
        def wrapped_function(*args, **kwargs):
            log_string = func.__name__ + " was called"
            print(log_string)
            # Open the logfile and append
            with open(logfile, 'a') as opened_file:
                # Now we log to the specified logfile
                opened_file.write(log_string + '\n')
            return func(*args, **kwargs)
        return wrapped_function
    return logging_decorator

@logit(logfile='func1.log')
def myfunc1():
    pass

In [60]:
myfunc1()

myfunc1 was called


In [61]:
myfunc1()

myfunc1 was called


And one more similar example follows, which displays the number of times a function is called. 

In [62]:
def count_calls(func):
    @functools.wraps(func)
    def wrapper_count_calls(*args, **kwargs):
        wrapper_count_calls.num_calls += 1
        print(f"Call {wrapper_count_calls.num_calls} of {func.__name__}")
        return func(*args, **kwargs)
    wrapper_count_calls.num_calls = 0
    return wrapper_count_calls

@count_calls
def say_hello():
    print("Hello!")

In [63]:
say_hello()

Call 1 of say_hello
Hello!


In [64]:
say_hello()

Call 2 of say_hello
Hello!


In [65]:
say_hello()

Call 3 of say_hello
Hello!


Decorators that can keep track of the state of a function, such as the number of times the function is called, are also referred to as ***stateful decorators***.

# 7.5 Class Decorators <a id="section5"/>

There are two different ways to use decorators with classes: decorate the methods of a class, or use classes as decorators.

## 7.5.1 Decorate the Methods of a Class

When we studied Python classes, we mentioned that the attributes can be *instance attributes* (that are assigned to particular class instances) and *class attributes* (that are assigned to all instances of a class). 

We also studied ***instance methods***, that are applied to class instances through the use of the keyword `self`. When an instance method is called, Python replaces the `self` argument with the instance object. The self parameter allows instance methods to access attributes and other methods in the class. 

In Python there are also ***class methods***, that are defined inside a class and are not connected to a particular instance of that class.

This is because sometimes programs need to process data associated with classes instead of instances. For instance, consider keeping track of the number of instances created from a class, or maintaining a list of all instances from a class that are currently in memory. This type of information and its processing are associated with the class rather than its instances. That is, the information is usually stored on the class itself and processed apart from any instance.

For such tasks, simple functions coded outside a class can often suffice. Because they can access class attributes through the class name, they have access to class data and never require access to an instance. However, to better associate such code with a class, and to allow such processing to be customized with inheritance as usual, it would be better to code these types of functions inside the class itself. To make this work, we use methods that  do not expect a `self` instance argument.

Commonly used decorators for class methods that are built-in in Python are `@classmethod` and `@staticmethod`.

The following code shows the difference in the syntax between instance method, `@classmethod` and `@staticmethod`. 

In [66]:
class MyClass:
    def instance_method(self, arg1, arg2, argN):
        return 'instance method called', self

    @classmethod
    def classmethod(cls, arg1, arg2, argN):
        return 'class method called', cls

    @staticmethod
    def staticmethod(arg1, arg2, argN):
        return 'static method called'

#### @staticmethod

In Python and other programming languages, a ***static method*** is a method that does not require the creation of an instance of a class. For Python, it means that the first argument of a static method is not `self`, but a regular positional or keyword argument. Also, a static method can have no arguments at all, as in the following example.

In general, static methods are used to create helper functions that have a logical connection with the class but do not have access to the attributes or methods of the class, or to the class instances.

In [67]:
class Cellphone:
    def __init__(self, brand, number):
        self.brand = brand
        self.number = number
        
    def get_number(self):
        return self.number
      
    @staticmethod
    def get_emergency_number():
        return "911"    

In [68]:
Cellphone.get_emergency_number()

'911'

In this example, `get_number()` is a regular instance method of the class and requires the creation of an instance. `get_emergency_number()` is a static method because it is decorated with the `@staticmethod` decorator. Also note that `get_emergency_number()` does not have `self` as the first argument, which means that it does not require the creation of an instance of the `Cellphone` class. 

Again, `get_emergency_number()` can just work as a standalone function, and it does not need to be defined as a static method. However, it makes sense and is intuitive to put it in the `Cellphone` class because a cellphone should be able to provide the emergency number.

Here is one more example of using a static method. The method `is_full_name()` just checks whether the entered name for a student consists of more than one string. 

In [69]:
class Student():

    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name

    @staticmethod
    def is_full_name(name_str):
        names = name_str.split(' ')
        return len(names) > 1

In [70]:
scott = Student('Scott',  'Robinson')

In [71]:
# call the static method
Student.is_full_name('Scott Robinson')

True

In [72]:
# call the static method
Student.is_full_name('Scott')          

False

And one more example of using `@staticmethod` follows. In order to convert the slash-dates to dash-dates, we used the function `toDashDate` within the `Dates` class. It is a static method because it doesn't need to access any properties of the class `Dates` through `self`. It is also possible to create a function `toDashDate()` outside the class, but since it works for dates, it is logical to keep it inside the `Dates` class.

In [73]:
class Dates:
    def __init__(self, date):
        self.date = date
        
    def getDate(self):
        return self.date

    @staticmethod
    def toDashDate(slash_date):
        return slash_date.replace("/", "-")

In [74]:
date1 = Dates("15/12/2016")
date1.getDate()

'15/12/2016'

In [75]:
date2 = Dates.toDashDate("15/12/2016")
date2

'15-12-2016'

In addition, static methods are used when we don't want subclasses of a superclass to change or override a specific implementation of a method. Because `@staticmethod` is ignorant of the class it is attached to, 
we can use it in subclasses just as it was defined in the superclass.

In the following code, `DatesWithSlashes` is derived from the superclass `Dates`. We wouldn't want the subclass `DatesWithSlashes` to override the static method `toDashDate()` because it only has a single use, i.e., change slash-dates to dash-dates. Therefore, we will use the static method to our advantage by overriding `getDate()` method in the subclass so that it works well with the `DatesWithSlashes` class.

In [76]:
class Dates:
    def __init__(self, date):
        self.date = date
        
    def getDate(self):
        return self.date

    @staticmethod
    def toDashDate(date):
        return date.replace("/", "-")

class DatesWithSlashes(Dates):
    def getDate(self):
        return Dates.toDashDate(self.date)

In [77]:
date1 = Dates("15/12/2016")
date1.getDate()

'15/12/2016'

In [78]:
date2 = DatesWithSlashes("15/12/2016")
date2.getDate()

'15-12-2016'

#### @classmethod

In Python, a ***class method*** is created with the `@classmethod` decorator and requires the class itself as the first argument, which is written as `cls`. A class method returns an instance of the class with supplied arguments or adds other additional functionality. 

In [79]:
class Cellphone:
    def __init__(self, brand, number):
        self.brand = brand
        self.number = number
        
    def get_number(self):
        return self.number
      
    @staticmethod
    def get_emergency_number():
        return "911"
      
    @classmethod
    def iphone(cls, number):
        print("An iPhone is created.")
        return cls("Apple", number)  

In [80]:
# create an iPhone instance using the class method
iphone = Cellphone.iphone("1112223333")

An iPhone is created.


In [81]:
# call the instance method
iphone.get_number()

'1112223333'

In [82]:
# call the static method
iphone.get_emergency_number()

'911'

In [83]:
samsung1 = Cellphone('Samsung', '123456789')

In [84]:
samsung1.get_number()

'123456789'

In [85]:
# the 'iphone' method cannot modify the instance 'samsung1'
samsung1.iphone('222222222')

An iPhone is created.


In [86]:
# the brand atribute of the instance was not modified by the 'iphone' method
# class method cannot modify specific instances
samsung1.brand

'Samsung'

In [87]:
# the number atribute of the instance was not modified by the 'iphone' method
samsung1.number

'123456789'

In this example, `iphone()` is a class method since it is decorated with the `@classmethod` decorator and has `cls` as the first argument. It returns an instance of the `Cellphone` class with the brand preset to `'Apple'`.

Class methods are often used as ***alternative constructors*** beside the `__init__()` constructor method, or as ***factory methods*** in order to create instances based on different use cases.

This is shown in the following example. Here, the `__init__()` constructor method takes two parameters `name` and `age`. The class method `fromBirthYear()` takes class, name, and birthYear, and calculates the current age by subtracting it from the current year. That is, it allows to create instances based on the year of birth, instead of based on the age. The reason for it is because we don’t want the list of arguments in the `__init__()` method  to be lengthy and confusing. Instead, we can use class methods to return a new instance based on different arguments. 

Note again that the `fromBirthYear()` method takes `Person` class as the first parameter `cls `, and not an instance of the class Person via `self`. Also, this method returns `cls(name, date.today().year - birthYear)`, which is equivalent to `Person(name, date.today().year - birthYear)`. 

In [88]:
from datetime import date

# random Person
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    @classmethod
    def fromBirthYear(cls, name, birthYear):
        return cls(name, date.today().year - birthYear)

    def display(self):
        print(self.name + "'s age is: " + str(self.age))

In [89]:
person1 = Person('Adam', 19)
person1.display()

Adam's age is: 19


In [90]:
person2 = Person.fromBirthYear('John',  1985)
person2.display()

John's age is: 37


In [91]:
# class method cannot modify specific instances
person1.fromBirthYear('John', 1985)

In [92]:
person1.name

'Adam'

The main difference between a static method and a class method is:

- Static methods can neither modify the class nor class instances, and it just deals with the attributes (i.e., arguments). They are used to create helper or utility functions. Static methods have a logical connection with the class but do not have access to class or instance states. 
- Class methods can modify the class since its parameter is always the class itself, but they cannot modify class instances. They can be used as factory methods to create new instances based on alternative information about a class.

One more example follows.

In [93]:
class Student:
    def __init__(self, name, grade, year, month, day):
        self.name = name;
        self.grade = grade;
        self.year = year;
        self.month = month;
        self.day = day;

    @classmethod
    def fromString(cls, name, grade, admission_date):
        year, month, day = admission_date.split("-")
        return cls(name, grade, year, month, day)

    @staticmethod
    def getRemarks(score):
        if score >=90 and score <=100:
            return "Excellent"
        elif score >= 80 and score < 90:
            return "Very Good"
        elif score >= 70 and score < 80:
            return "Good"
        elif score >= 60 and score < 70:
            return "Keep it up"
        elif score >= 0 and score < 60:
            return "Improve"
        else:
            return "Invalid Score"

    def displayInformation(self):
        print(f"Name: {self.name}, Grade: {self.grade}")
        print(f"Date of Admission: Year: {self.year}, Month: {self.month}, Day: {self.day}")

The class method `fromString()` above allows to create new `Student` instances by using name, grade, and admission date, instead of name, grade, year, month, and day. The static method `getRemarks()` takes the score of a student and outputs the remarks. This method has a logical connection with the `Student` class but does not use its attributes or methods.

In [94]:
student1 = Student("Ashton", 10, 2005, 7, 1)
student1.displayInformation()

Name: Ashton, Grade: 10
Date of Admission: Year: 2005, Month: 7, Day: 1


In [95]:
student2 = Student.fromString("Alice", 9, "2008-1-15")
student2.displayInformation()

Name: Alice, Grade: 9
Date of Admission: Year: 2008, Month: 1, Day: 15


In [96]:
student1.getRemarks(95)

'Excellent'

To repeat, class method cannot access an instance attributes or call regular instance methods. Class methods can only call other class methods or access class attributes.

Regarding class inheritance, whenever we derive a subclass from a superclass that implements a class method, this ensures correct instance creation of the derived class. On the other hand, a static method does not change in a derived subclass.

In [97]:
from datetime import date

# random Person
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    # a static method to create a Person object by father's age and age difference   
    @staticmethod
    def fromFathersAge(name, fatherAge, fatherPersonAgeDiff):
        return Person(name, date.today().year - fatherAge + fatherPersonAgeDiff)

    # a class method to create a Person object by birth year
    @classmethod
    def fromBirthYear(cls, name, birthYear):
        return cls(name, date.today().year - birthYear)

    def display(self):
        print(self.name + "'s age is: " + str(self.age))
        
    # a static method to check if a Person is adult or not
    @staticmethod
    def isAdult(age):
        return age > 18

class Man(Person):
    sex = 'Male'

In this example, using a static method to create a class instance causes a problem when inheriting `Person` to `Man`, since `fromFathersAge()` method doesn't return a Man object but an object from its base class Person. Using a class method as `fromBirthYear()` can ensure the inheritance, since it takes the class itself as the first parameter in the method.

In [98]:
man1 = Man.fromBirthYear('John', 1985)
man1.display()

John's age is: 37


In [99]:
Person.isAdult(25)

True

In [100]:
# man2 is not an instance of Man, because we used the class method
print(isinstance(man1, Man))

True


In [101]:
man2 = Man.fromFathersAge('John', 1965, 20)
man2.display()

John's age is: 77


In [102]:
# man2 is not an instance of Man, because we used the static method
print(isinstance(man2, Man))

False


In [103]:
print(isinstance(man2, Person))

True


## 7.5.2 Using Classes as Decorators

So far we have studied ***function decorators***, but Python also supports using ***class decorators***.

Recall that the decorator syntax `@my_decorator` is just an easier way of writing `func = my_decorator(func)`. Therefore, if `my_decorator` is a class, it needs to take `func` as an argument in its `.__init__()` method. 

Furthermore, the class instance needs to be callable so that it can stand in for the decorated function `func`. In general, all callable objects in Python have implemented the special method `.__call__()`. Hence, the special method `.__call__()` is used in class decorators to make the instances of that class callable. 

In the following example, the class `CountCalls` is similar to the previous `count_calls()` decorator in the Logging examples section, and it is used to count the numbers a function is called. The class decorator `@CountCalls` uses the same syntax with the symbol `@`.

In [104]:
class CountCalls:
    def __init__(self, func):
        self.func = func
        self.num_calls = 0

    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"Call {self.num_calls} of {self.func.__name__!r}")
        return self.func(*args, **kwargs)

@CountCalls
def say_hello():
    print("Hello!")

In [105]:
say_hello()

Call 1 of 'say_hello'
Hello!


In [106]:
say_hello()

Call 2 of 'say_hello'
Hello!


This is another example related to the example for creating a logfile in the Logging section, which is extended so that the logit function is rebuilt as a logit class. Note again to use of `func` in the `.__init__()` method, and the `.__call__()` method. 

In [110]:
class logit():

    logfile = 'out.log'

    def __init__(self, func):
        self.func = func

    def __call__(self, *args):
        log_string = self.func.__name__ + " was called"
        print(log_string)
        # Open the logfile and append
        with open(self.logfile, 'a') as opened_file:
            # Now we log to the specified logfile
            opened_file.write(log_string + '\n')
        # Now, send a notification
        self.notify()

        # return base func
        return self.func(*args)
    
    def notify(self):
         # Will not be implemented here
        pass

This implementation has an advantage of being cleaner than the nested function approach, and wrapping a function still uses the same syntax as before.

In [111]:
logit.logfile = 'func2.log' 

@logit
def myfunc2():
    pass

In [112]:
myfunc2()

myfunc2 was called


Now, let’s subclass `logit` to add email functionality. `@email_logit` works just like `@logit` but sends an email to the admin in addition to logging.

In [113]:
class email_logit(logit):
    '''
    A logit implementation for sending emails to admins
    when the function is called.
    '''
    def __init__(self, email='admin@myproject.com', *args, **kwargs):
        self.email = email
        super(email_logit, self).__init__(*args, **kwargs)
        
    def notify(self):
        # Send an email to self.email
        # Will not be implemented here
        pass

# References <a id="section6"/>

1. Primer on Python Decorators, available at: [https://realpython.com/primer-on-python-decorators/](https://realpython.com/primer-on-python-decorators/).
2. Python Decorators at Programiz, available at: [https://www.programiz.com/python-programming/decorator](https://www.programiz.com/python-programming/decorator).
2. Mark Lutz, "Learning Python," 5-th edition, O-Reilly, 2013. ISBN: 978-1-449-35573-9.
3. Pierian Data Inc., "Complete Python 3 Bootcamp," codes available at: [https://github.com/Pierian-Data/Complete-Python-3-Bootcamp](https://github.com/Pierian-Data/Complete-Python-3-Bootcamp).
5. Lynn Kwong, How to Use the Magical @staticmethod, @classmethod, and @property Decorators in Python, available at: [https://betterprogramming.pub/how-to-use-the-magical-staticmethod-classmethod-and-property-decorators-in-python-e42dd74e51e7]()
6. Class Method, Static Method at Programiz, available at: [https://www.programiz.com/python-programming/methods/built-in/classmethod](https://www.programiz.com/python-programming/methods/built-in/classmethod)
7. Python Tips, Decorators, available at: [https://book.pythontips.com/en/latest/decorators.html](https://book.pythontips.com/en/latest/decorators.html)